In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [3]:
# load traffic data
traffic = pd.read_csv('LE5D_D.txt', sep='\t')
#print(type(traffic))


# Define the features variables
features = ['tt_ex','nsnow', 'ntemp']
#print(traffic[features])
# convert DataFrame to narray
features_array=traffic[features].values
print(features_array)



# Define target variables
target = 'tt_vf'
#print(traffic[target])
# convert DataFrame to narray
target_array=traffic[target].values
print(target_array)


snow_array=traffic['snow'].values
print(snow_array)


temp_array=traffic['temp'].values
print(temp_array)

#traffic['nsnow'].value_counts()
#traffic['ntemp'].value_counts()

[[  0.75199712   1.         -21.4       ]
 [  0.64276338   0.         -26.2       ]
 [  0.79148195   0.         -25.8       ]
 ...
 [  0.84395048   0.6        -15.4       ]
 [  0.90538953   3.         -20.3       ]
 [  0.8232439    0.         -28.6       ]]
[0.49946938 0.65257516 0.75219117 ... 0.87688489 0.8935321  0.8232439 ]
['SC2' 'SC1' 'SC1' ... 'SC2' 'SC2' 'SC1']
['CC6' 'CC7' 'CC7' ... 'CC5' 'CC6' 'CC7']


In [8]:
kfold = KFold(n_splits=10)
# kfold.split(X)는 폴드 세트를 3번 반복할 때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환. 
n_iter =0
for train_index, test_index  in kfold.split(traffic):
    n_iter += 1
    label_train= traffic['temp'].iloc[train_index] 
    label_test= traffic['temp'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 CC1    997
CC2    203
CC3    182
CC4    102
CC5     78
CC6     44
CC7     37
Name: temp, dtype: int64
검증 레이블 데이터 분포:
 CC1    106
CC3     23
CC2     19
CC4     16
CC7      7
CC6      6
CC5      6
Name: temp, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 CC1    964
CC2    199
CC3    198
CC4    113
CC5     77
CC6     49
CC7     43
Name: temp, dtype: int64
검증 레이블 데이터 분포:
 CC1    139
CC2     23
CC3      7
CC5      7
CC4      5
CC6      1
CC7      1
Name: temp, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 CC1    1004
CC2     195
CC3     173
CC4     104
CC5      74
CC6      50
CC7      43
Name: temp, dtype: int64
검증 레이블 데이터 분포:
 CC1    99
CC3    32
CC2    27
CC4    14
CC5    10
CC7     1
Name: temp, dtype: int64
## 교차 검증: 4
학습 레이블 데이터 분포:
 CC1    988
CC2    205
CC3    177
CC4    106
CC5     80
CC6     46
CC7     41
Name: temp, dtype: int64
검증 레이블 데이터 분포:
 CC1    115
CC3     28
CC2     17
CC4     12
CC5      4
CC6      4
CC7      3
Name: temp, dtype: int64
## 교차 검증: 5
학습 레이블

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10)
n_iter=0

for train_index, test_index in skf.split(features_array, temp_array): # works traffic, traffic['temp'] as well
    
    n_iter += 1
    print(train_index)
    print(test_index)
    
    label_train= traffic['temp'].iloc[train_index]
    label_test= traffic['temp'].iloc[test_index]
    
    train_size = label_train.shape[0]
    test_size = label_test.shape[0]
    
   

    print('## 교차 검증: {0}, 학습 데이터 크기: {1}, 검증 데이터 크기: {2} '.format(n_iter, train_size, test_size))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    
    
   # label_train_1= traffic['tt_vf'].iloc[train_index]
   # label_test_1= traffic['tt_vf'].iloc[test_index]
   # print('## 교차 검증: {0}'.format(n_iter))
   # print('학습 레이블 데이터 분포:\n', label_train_1.value_counts())
   # print('검증 레이블 데이터 분포:\n', label_test_1.value_counts())
    
    

In [7]:
dt = DecisionTreeRegressor(random_state=156)

skfold = StratifiedKFold(n_splits=10)
n_iter=0
cv_accuracy=[]

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  
for train_index, test_index  in skfold.split(features_array, temp_array):
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features_array[train_index], features_array[test_index]
    y_train, y_test = target_array[train_index], target_array[test_index]
    
    #학습 및 예측 
    dt.fit(X_train , y_train)    
    pred = dt.predict(X_test)

    # 반복 시 마다 정확도 측정 
    n_iter += 1
    accuracy = np.round(mean_squared_error(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 학습 세트 인덱스:{1}'.format(n_iter,train_index))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산 
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :0.0118, 학습 데이터 크기: 1643, 검증 데이터 크기: 183
#1 학습 세트 인덱스:[  13   14   37 ... 1823 1824 1825]
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  78  79  82  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 293 300 304
 334 335 336]

#2 교차 검증 정확도 :0.0151, 학습 데이터 크기: 1643, 검증 데이터 크기: 183
#2 학습 세트 인덱스:[   0    1    2 ... 1823 1824 1825]
#2 검증 세트 인덱스:[ 13  14  37  75  76  